In [2]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.5 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import random
import torch
import unicodedata

random.seed(42)

In [4]:
model_name = "allegro/herbert-base-cased"
device = 'cuda'

tokenizer_herbert = AutoTokenizer.from_pretrained(model_name)
model_herbert = AutoModel.from_pretrained(model_name).to(device)

text = 'Bardzo lubię lody malinowe z bitą śmietaną.'

token_ids = tokenizer_herbert(text, return_tensors='pt')['input_ids'][0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
model_name_papuga = 'flax-community/papuGaPT2'
tokenizer_papuga = AutoTokenizer.from_pretrained(model_name_papuga)
model_papuga = AutoModelForCausalLM.from_pretrained(model_name_papuga).to(device)

tokenizer_config.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/888k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [6]:
emb_papuga = model_papuga.transformer.wte.weight.detach().to(device)

In [7]:
input_ids = tokenizer_papuga("kwiaciarnia", return_tensors='pt')['input_ids'].to(device)


In [8]:
model_herbert.embeddings.word_embeddings(input_ids)[0, :, :].mean(dim=0).shape

torch.Size([768])

In [36]:
def generate_embedding(word, context):
    tokenizer = tokenizer_herbert if context else tokenizer_papuga
    input_ids = tokenizer(word, return_tensors='pt')['input_ids'].to(device)
    e = None
    if context:
        outputs = model_herbert(input_ids=input_ids)
        # e = outputs.last_hidden_state.detach().to(device)[0, :, :].sum(dim=0)
        e = model_herbert.embeddings.word_embeddings(input_ids)[0, :, :].mean(dim=0)
        # e = outputs.last_hidden_state.detach().to(device)[0, :, :].mean(dim=0)

        # e = outputs.last_hidden_state.detach().to(device)[0, 0, :]
    else:
        # e = emb_papuga[input_ids][0, :, :].sum(dim=0)
        e = emb_papuga[input_ids][0, 0, :]

        # e = emb_papuga[input_ids][0, 0, :]

    return e

In [10]:
e1 = generate_embedding("kwiaciarnia", True)

In [11]:
e2 = generate_embedding("kwiaciarnia", False)

In [12]:
clusters_txt = '''
piśmiennicze: pisak flamaster ołówek długopis pióro
małe_ssaki: mysz szczur chomik łasica kuna bóbr
okręty: niszczyciel lotniskowiec trałowiec krążownik pancernik fregata korweta
lekarze: lekarz pediatra ginekolog kardiolog internista geriatra
zupy: rosół żurek barszcz
uczucia: miłość przyjaźń nienawiść gniew smutek radość strach
działy_matematyki: algebra analiza topologia logika geometria
budynki_sakralne: kościół bazylika kaplica katedra świątynia synagoga zbór
stopień_wojskowy: chorąży podporucznik porucznik kapitan major pułkownik generał podpułkownik
grzyby_jadalne: pieczarka borowik gąska kurka boczniak kania
prądy_filozoficzne: empiryzm stoicyzm racjonalizm egzystencjalizm marksizm romantyzm
religie: chrześcijaństwo buddyzm islam prawosławie protestantyzm kalwinizm luteranizm judaizm
dzieła_muzyczne: sonata synfonia koncert preludium fuga suita
cyfry: jedynka dwójka trójka czwórka piątka szóstka siódemka ósemka dziewiątka
owady: ważka biedronka żuk mrówka mucha osa pszczoła chrząszcz
broń_biała: miecz topór sztylet nóż siekiera
broń_palna: karabin pistolet rewolwer fuzja strzelba
komputery: komputer laptop kalkulator notebook
kolory: biel żółć czerwień błękit zieleń brąz czerń
duchowny: wikary biskup ksiądz proboszcz rabin pop arcybiskup kardynał pastor
ryby: karp śledź łosoś dorsz okoń sandacz szczupak płotka
napoje_mleczne: jogurt kefir maślanka
czynności_sportowe: bieganie skakanie pływanie maszerowanie marsz trucht
ubranie:  garnitur smoking frak żakiet marynarka koszula bluzka sweter sweterek sukienka kamizelka spódnica spodnie
mebel: krzesło fotel kanapa łóżko wersalka sofa stół stolik ława
przestępca: morderca zabójca gwałciciel złodziej bandyta kieszonkowiec łajdak łobuz
mięso_wędliny wieprzowina wołowina baranina cielęcina boczek baleron kiełbasa szynka schab karkówka dziczyzna
drzewo: dąb klon wiąz jesion świerk sosna modrzew platan buk cis jawor jarzębina akacja
źródło_światła: lampa latarka lampka żyrandol żarówka reflektor latarnia lampka
organ: wątroba płuco serce trzustka żołądek nerka macica jajowód nasieniowód prostata śledziona
oddziały: kompania pluton batalion brygada armia dywizja pułk
napój_alkoholowy: piwo wino wódka dżin nalewka bimber wiśniówka cydr koniak wiśniówka
kot_drapieżny: puma pantera lampart tygrys lew ryś żbik gepard jaguar
metal: żelazo złoto srebro miedź nikiel cyna cynk potas platyna chrom glin aluminium
samolot: samolot odrzutowiec awionetka bombowiec myśliwiec samolocik helikopter śmigłowiec
owoc: jabłko gruszka śliwka brzoskwinia cytryna pomarańcza grejpfrut porzeczka nektaryna
pościel: poduszka prześcieradło kołdra kołderka poduszeczka pierzyna koc kocyk pled
agd: lodówka kuchenka pralka zmywarka mikser sokowirówka piec piecyk piekarnik
'''

In [13]:
def modify_string(input_string):
    s = "".join(
        char for char in unicodedata.normalize("NFD", input_string)
        if unicodedata.category(char) != "Mn"
    )
    if len(s) < 2:
        return s
    return s[:-2] + s[-1] + s[-2]

In [14]:
modify_string("żółć")

'zocł'

In [15]:
C = {}
words = set()


for x in clusters_txt.split('\n'):
    L = x.split()
    if len(L) < 2:
        continue
    C[L[0]] = L[1:]
    words.update(L[1:])

groups = list(C.keys())

In [16]:
groups

['piśmiennicze:',
 'małe_ssaki:',
 'okręty:',
 'lekarze:',
 'zupy:',
 'uczucia:',
 'działy_matematyki:',
 'budynki_sakralne:',
 'stopień_wojskowy:',
 'grzyby_jadalne:',
 'prądy_filozoficzne:',
 'religie:',
 'dzieła_muzyczne:',
 'cyfry:',
 'owady:',
 'broń_biała:',
 'broń_palna:',
 'komputery:',
 'kolory:',
 'duchowny:',
 'ryby:',
 'napoje_mleczne:',
 'czynności_sportowe:',
 'ubranie:',
 'mebel:',
 'przestępca:',
 'mięso_wędliny',
 'drzewo:',
 'źródło_światła:',
 'organ:',
 'oddziały:',
 'napój_alkoholowy:',
 'kot_drapieżny:',
 'metal:',
 'samolot:',
 'owoc:',
 'pościel:',
 'agd:']

In [37]:
def create_embeddings(C, context):
    v = {}
    for c in C.values():
        for w in c:
            v[w] = generate_embedding(w, context)
    return v

In [38]:
v_context = create_embeddings(C, True)
v_context_free = create_embeddings(C, False)

In [19]:
def create_modifications(C):
    v = {}
    for c in C.values():
        for w in c:
            v[w] = modify_string(w)
    return v

In [20]:
def create_embeddings_modifications(C, context):
    v = {}
    for w in C.values():
        v[w] = generate_embedding(w, context)
    return v

In [21]:
v_modifications = create_modifications(C)
v_modifications_emb_context = create_embeddings_modifications(v_modifications, True)
v_modifications_emb_context_free = create_embeddings_modifications(v_modifications, False)

In [22]:
def choose_another(A, a):
    while True:
        b = random.choice(A)
        if a != b:
            return b

In [23]:
def evaluate(vectors, N = 30000, modifications = False):
    score = 0.0
    for i in range(N):
        g1 = random.choice(groups)
        A = C[g1]
        B = C[choose_another(groups, g1)]

        a1 = random.choice(A)
        a2 = choose_another(A, a1)
        b = random.choice(B)

        if modifications:
            a1 = v_modifications[a1]
            a2 = v_modifications[a2]
            b = v_modifications[b]

        #print (a1,a2,b)
        if not a1 in vectors or not a2 in vectors or not b in vectors:
            score += 0.5
            continue

        va1 = vectors[a1]
        va2 = vectors[a2]
        vb  = vectors[b]
        # print(va1, va2, vb)
        # return

        if va1.dot(va2) > va1.dot(vb):
            score += 1.0
        else:
            pass
            #print (a1,a2,'--',b)

    print ('TOTAL SCORE:', score/N)

In [31]:
evaluate(v_context)

TOTAL SCORE: 0.6410333333333333


In [39]:
evaluate(v_context_free)

TOTAL SCORE: 0.5441


In [26]:
evaluate(v_modifications_emb_context, modifications=True)

TOTAL SCORE: 0.5812


In [27]:
evaluate(v_modifications_emb_context_free, modifications=True)

TOTAL SCORE: 0.5475333333333333
